In [105]:
import pickle
import numpy
with open("aa_sim_array.pickle", 'rb') as f:
    aa_mat = pickle.load(f)
with open("pos_sim_array.pickle", 'rb') as f:
    str_mat = pickle.load(f)
with open("pos_pdb_ids.pickle", 'rb') as f:
    ids = pickle.load(f)
with open("pos_labels.pickle", 'rb') as f:
    labels = pickle.load(f)

In [43]:
str_mat #Levenshtein distances - the bigger the less similar

array([[0.        , 0.41605839, 0.29927007, ..., 1.5212766 , 1.49468085,
        1.50531915],
       [0.        , 0.        , 0.41586538, ..., 1.53191489, 1.52659574,
        1.5212766 ],
       [0.        , 0.        , 0.        , ..., 1.56382979, 1.56382979,
        1.56382979],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.18617021,
        0.16489362],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.07978723],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [44]:
aa_mat #Levenshtein distances - the bigger the less similar

array([[0.        , 0.00235849, 0.00707547, ..., 1.65829146, 1.55399061,
        1.65829146],
       [0.        , 0.        , 0.00471698, ..., 1.65829146, 1.55868545,
        1.65829146],
       [0.        , 0.        , 0.        , ..., 1.65829146, 1.55868545,
        1.65829146],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.11557789,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.11557789],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [45]:
#interested in big aa difference, but small str difference -> aa_mat / str_mat -> the bigger the better (more interesting)

str_mat = str_mat + 0.00001
res = aa_mat / str_mat

In [46]:
import numpy as np
res = np.array(res)
np.max(res)

2127.659574468085

In [47]:
import bottleneck as bn

def top_n_indexes(arr, n):
    idx = bn.argpartition(arr, arr.size-n, axis=None)[-n:]
    width = arr.shape[1]
    return [divmod(i, width) for i in idx]

In [80]:
idx = top_n_indexes(res, 3000)
idx.sort(key = lambda tup: tup[0])

In [56]:
temp = np.partition(-res.flatten(), 5)
-temp[:5]

array([2127.65957447, 2127.65957447,   11.73043078,  513.69863014,
         10.94649216])

In [61]:
#sanity check
for i in idx:
    print(res[i[0], i[1]])

11.730430777127433
2127.659574468085
2127.659574468085
513.6986301369863
10.946492161978345


In [107]:
#For twilight zone, we're only interested if aa distance is bigger than 0.75
count = 0
important_ids = []
important_labels = []
for i in idx:
    if (aa_mat[i[0], i[1]] > 0.7) and (str_mat[i[0], i[1]] < 0.25):
        important_ids.append(ids[i[0]])
        important_ids.append(ids[i[1]])
        important_labels.append(labels[i[0]])
        important_labels.append(labels[i[1]])
        count = count + 1
        print('"' + ids[i[0]] + '"' + ' paired with ' + '"' + ids[i[1]] + '"' + ' yields ' + str(res[i[0], i[1]]) + ' with aa distance ' + str(aa_mat[i[0], i[1]]) + ' and labels ' + labels[i[0]] + ' and ' + labels[i[1]])

"4PKM A 38:278" paired with "6RB9 A 34:260" yields 4.456516751916893 with aa distance 0.8152610441767069 and labels pos_18_ETX_MTX2.txt and pos_18_ETX_MTX2.txt
"4QX0 A 509:644" paired with "2L38 A 1:175" yields 3.9885376765703473 with aa distance 0.7377049180327869 and labels pos_22_Thiol_cytolysin.txt and pos_22_Thiol_cytolysin.txt
"4QX1 A 509:644" paired with "2L38 A 1:175" yields 3.0018631932819995 with aa distance 0.734251968503937 and labels pos_22_Thiol_cytolysin.txt and pos_22_Thiol_cytolysin.txt
"4QX2 A 509:644" paired with "2L38 A 1:175" yields 3.292018219862203 with aa distance 0.7591836734693878 and labels pos_22_Thiol_cytolysin.txt and pos_22_Thiol_cytolysin.txt
"4QX3 A 509:644" paired with "2L38 A 1:175" yields 3.2290276390877612 with aa distance 0.75 and labels pos_22_Thiol_cytolysin.txt and pos_22_Thiol_cytolysin.txt
"1JI6 A 512:651" paired with "2L38 A 1:175" yields 3.4528113309634754 with aa distance 0.75 and labels pos_22_Thiol_cytolysin.txt and pos_22_Thiol_cytolysin

In [108]:
count

99

In [109]:
np.unique(np.array(important_ids))

array(['1JI6 A 512:651', '1O72 A 1:175', '1T5R A 29:275',
       '1XEZ A 190:452', '2I88 A 345:519', '2L38 A 1:175',
       '2LKF A 34:284', '2QQH A 237:460', '2RD7 A 237:460',
       '2ZS6 B 227:423', '3LIM A 3:178', '3LKF A 34:284',
       '3O44 A 190:452', '3OJY A 244:460', '3OJY B 219:442',
       '3ROH A 57:302', '3VWI A 3:178', '3ZWG A 3:178', '4E0S B 270:493',
       '4EN6 B 227:423', '4EN7 B 227:423', '4EN8 B 227:423',
       '4IYA A 29:275', '4IYC A 29:275', '4IZL A 29:275', '4MKO A 43:246',
       '4MKQ A 43:246', '4P1X B 29:275', '4PKM A 38:278',
       '4QX0 A 509:644', '4QX1 A 509:644', '4QX2 A 509:644',
       '4QX3 A 509:644', '5J67 A 802:916', '5J68 A 802:916',
       '5J69 A 802:915', '5J69 B 802:916', '5K59 C 62:305',
       '6FBM A 62:303', '6H03 B 291:514', '6H03 D 227:448',
       '6H03 F 237:460', '6H04 B 291:514', '6H04 D 227:448',
       '6H04 F 237:460', '6OQ6 A 1090:1431', '6RB9 A 34:260'],
      dtype='<U16')

In [110]:
len(np.unique(np.array(important_ids)))

47

In [111]:
np.unique(np.array(important_labels))

array(['pos_18_ETX_MTX2.txt', 'pos_22_Thiol_cytolysin.txt',
       'pos_37_Leukocidin.txt', 'pos_4_T4BSS_DotI_IcmL.txt',
       'pos_9_Bac_thur_toxin.txt'], dtype='<U26')